Below a way to compute the minimum population sufficient to elect the president of the US, solving this optimization (minimization) 
problem using Google or-tools solver to find the minimum amount of voters necessary to win 170 electoral college seats
in the 2016 elections.

source for the solver implementation:
https://stackoverflow.com/questions/69473985/pandas-find-subset-of-rows-minimizing-the-sum-of-a-column-under-other-column-con?noredirect=1&lq=1

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations, chain

In [2]:
#load data
els = pd.read_excel('elections.xlsx')

In [3]:
%%time

#implement solution

# Google or-tools solver
from ortools.sat.python import cp_model

import timeit

def solve(df, threshold):
    '''
    Uses or-tools module to solve optimization

    '''
    weights = df['Target']
    cost = df['Cost']
    names = df['Names']

    # Creates the model.
    model = cp_model.CpModel()

    # Step 1: Create the variables
    # array containing row selection flags i.e. True if row k is selected, False otherwise
    # Note: treated as 1/0 in arithmeetic expressions
    row_selection = [model.NewBoolVar(f'{i}') for i in range(df.shape[0])]

    # Step 2: Define the constraints
    # The sum of the weights for the selected rows should be >= threshold
    model.Add(weights.dot(row_selection) >= threshold)

    # Step 3: Define the objective function
    # Minimize the total cost (based upon rows selected)
    model.Minimize(cost.dot(row_selection))

    # Step 4: Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.Solve(model)

    # Get the rows selected
    rows = [row for row in range(df.shape[0]) if solver.Value(row_selection[row])]

    return names[rows]


# Setup
df = els.copy().rename(columns={'State':'Names', 'Population':'Cost', 'EC':'Target'})

Wall time: 953 ms


In [4]:
#sum of EC seats in solution
ecs = df[df['Names'].isin(solve(df, 270))]['Target'].sum()

#sum of population in solution
pop = df[df['Names'].isin(solve(df, 270))]['Cost'].sum()

#percentage of population represented by solution
share = df[df['Names'].isin(solve(df, 270))]['Cost'].sum() / df['Cost'].sum()

#electoral colleges in solution
ec = solve(df, 270)

In [5]:
print('Summary of optimization model solution: ')
print('Electoral college seats: ', ecs)
print('Voters: ', pop)
print('Voters out of total population: ', round(share, 2))
print('Electoral colleges: \n', ec)

Summary of optimization model solution: 
Electoral college seats:  270
Voters:  134737835
Voters out of total population:  0.44
Electoral colleges: 
 0            Wyoming
1               D.C.
2            Vermont
3       North Dakota
4             Alaska
5       South Dakota
6           Delaware
7            Montana
8       Rhode Island
9      New Hampshire
10             Maine
11            Hawaii
12             Idaho
13          Nebraska
14     West Virginia
15        New Mexico
16            Nevada
17              Utah
18            Kansas
19          Arkansas
20       Mississippi
21              Iowa
22       Connecticut
23          Oklahoma
24            Oregon
25          Kentucky
26         Louisiana
27    South Carolina
28           Alabama
29          Colorado
30         Minnesota
31         Wisconsin
32          Maryland
33          Missouri
34         Tennessee
35           Arizona
36           Indiana
37     Massachusetts
38        Washington
41    North Carolina
Name: Name